In [1]:
import pandas as pd

In [5]:
import pronouncing

In [36]:
import random

In [2]:
pd.read_csv("authors.csv")

,code,name,birth_year,death_year,title,first_line,image_url,poem_link
0,APH,Alexander Pope,1688,1744.0,An Essay on Man: Epistle I,Hope springs eternal in the human breast:\nMan...,https://upload.wikimedia.org/wikipedia/commons...,https://www.poetryfoundation.org/poems/44899/a...
1,AT,Alfred Lord Tennyson,1809,1892.0,In Memoriam A.H.H. 1849,The heart that never plighted troth\n But...,https://upload.wikimedia.org/wikipedia/commons...,NaN
2,ALT,Alfred Lord Tennyson,1809,1892.0,"In Memoriam, [Ring out, wild bells]","The year is dying in the night;\nRing out, wil...",https://upload.wikimedia.org/wikipedia/commons...,https://poets.org/poem/memoriam-ring-out-wild-...
3,RBU,Robert Burns,1759,1796.0,Auld Lang Syne,"Should auld acquaintance be forgot,\nAnd never...",https://poets.org/sites/default/files/images/b...,https://poets.org/poem/auld-lang-syne
4,ESR,Edna St. Vincent Millay,1892,1950.0,Recuerdo,"We were very tired, we were very merry—\nWe ha...",https://upload.wikimedia.org/wikipedia/commons...,https://www.poetryfoundation.org/poetrymagazin...
...,...,...,...,...,...,...,...,...
95,LYL,Li-Young Lee,1957,NaN,Persimmons,In sixth grade Mrs. Walker\nslapped the back o...,https://upload.wikimedia.org/wikipedia/commons...,NaN
96,WAM,W.H. Auden,1907,1973.0,In Memory of W. B. Yeats,You were silly like us; your gift survived it ...,https://upload.wikimedia.org/wikipedia/commons...,NaN
97,RHA,Robert Hayden,1913,1980.0,Those Winter Sundays,Sundays too my father got up early\nand put hi...,https://upload.wikimedia.org/wikipedia/en/4/4d...,NaN
98,THG,Ted Hughes,1930,1998.0,The Thought-Fox,"Across clearings, an eye,\nA widening deepenin...",https://upload.wikimedia.org/wikipedia/en/thum...,NaN


In [3]:
df = pd.read_csv("authors.csv")

In [4]:
df['first_line']

0     Hope springs eternal in the human breast:\nMan...
1     The heart that never plighted troth\n      But...
2     The year is dying in the night;\nRing out, wil...
3     Should auld acquaintance be forgot,\nAnd never...
4     We were very tired, we were very merry—\nWe ha...
                            ...                        
95    In sixth grade Mrs. Walker\nslapped the back o...
96    You were silly like us; your gift survived it ...
97    Sundays too my father got up early\nand put hi...
98    Across clearings, an eye,\nA widening deepenin...
99    i’ve left Earth in search of darker planets, a...
Name: first_line, Length: 100, dtype: object

In [58]:
pd.options.display.max_colwidth = 5000

In [59]:
df['first_line'].head(3)

0    Hope springs eternal in the human breast:\nMan never is, but always to be blest:\nThe soul, uneasy and confin'd from home,\nRests and expatiates in a life to come.\nLo! the poor Indian, whose untutor'd mind\nSees God in clouds, or hears him in the wind;\nHis soul, proud science never taught to stray\nFar as the solar walk, or milky way;
1                                                                                      The heart that never plighted troth\n      But stagnates in the weeds of sloth,\nNor any want-begotten rest.\nI hold it true, whate’er befall,\n      I feel it, when I sorrow most;\n      ‘Tis better to have loved and lost\nThan never to have loved at all.
2                                                                                                                       The year is dying in the night;\nRing out, wild bells, and let him die.\n\nRing out the old, ring in the new,\n   Ring, happy bells, across the snow:\n   The year is going, let him go;\nRing o

In [63]:
df['first_line'].apply(get_rhymes)

0                                                                                                                                                                                                                                                                       ([breast:, blest:, home,, come., mind, wind;, stray, way;], [[molest:, infest:, unaddressed:, attest:, impressed:], [infest:, detest:, prest:, natwest:, confessed:], [strome,, shalom,, gloam,, noam,, boehm,], [humm., thum., dum., pflum., alum.], [blind, undermined, fined, defined, unkind], [resigned;, mankind;, disinclined;, reclined;, twined;], [monterey, seay, convey, pei, deseret], [voisey;, gai;, monet;, dekay;, attache;]])
1                                                                                                                                                                                                                                                                                                               

In [75]:
import nltk
nltk.download("brown")


[nltk_data] Downloading package brown to /Users/melwalsh/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [80]:
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/melwalsh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [81]:
import pandas as pd
import random
import re
import pronouncing
from nltk.corpus import brown
from nltk import pos_tag
from collections import Counter

random.seed(42)  # reproducible builds


# --------- CONFIG ---------
MIN_FREQ = 50  # 👈 raise/lower this as you like for "common" words


# --------- helpers for cores, POS & frequency ---------
def word_core_lower(word: str) -> str:
    """
    Very forgiving core extractor for duplicate checks:
    - takes the first run of letters/apostrophes
    - lowercases it
    So 'breast:', 'Breast', '"breast:"' -> 'breast'
    """
    m = re.search(r"[A-Za-z']+", word)
    if m:
        return m.group(0).lower()
    return word.lower()


# Build a frequency counter over the Brown corpus
word_freq = Counter(w.lower() for w in brown.words())

def is_common_word(word: str, min_frequency: int = MIN_FREQ) -> bool:
    """
    Check if word appears frequently enough in common English,
    using the Brown corpus as a rough filter.
    Also require the core to be at least 3 letters long.
    """
    core = word_core_lower(word)
    if len(core) < 3:
        return False
    return word_freq.get(core, 0) >= min_frequency


def slugify(s: str) -> str:
    """Make a URL / id-friendly slug from a string."""
    s = s.lower()
    s = re.sub(r"[^\w\s-]", "", s)  # remove punctuation
    s = re.sub(r"\s+", "-", s).strip("-")  # spaces -> hyphens
    return s


def split_core_and_punct(word: str):
    """
    Split token into alphabetic core + trailing punctuation.
    Examples:
        'night,'    -> ('night', ',')
        'Sea!'      -> ('Sea', '!')
        'breast:'   -> ('breast', ':')
        '"time."'   -> ('time', '."')
    """
    word = word.strip()
    m = re.match(r"^([A-Za-z']+)([^A-Za-z']*)$", word)
    if m:
        return m.group(1), m.group(2)
    else:
        return word, ""


def coarse_pos(tag: str) -> str:
    """
    Map a fine-grained Penn Treebank POS tag to a coarse tag.
    """
    if tag.startswith("N"):
        return "NOUN"
    if tag.startswith("V"):
        return "VERB"
    if tag.startswith("J"):
        return "ADJ"
    if tag.startswith("R"):
        return "ADV"
    return "OTHER"


def get_coarse_pos(word: str) -> str | None:
    """
    Get a coarse POS tag for a single word using NLTK's pos_tag.
    Returns None if tagging fails.
    """
    core = word_core_lower(word)
    if not core:
        return None
    try:
        tag = pos_tag([core])[0][1]
        return coarse_pos(tag)
    except Exception:
        return None


# --------- rhyme distractors ---------
def rhyme_distractors(core: str,
                      punct: str,
                      forbidden_core_set=None,
                      max_distractors: int = 5,
                      target_pos: str | None = None):
    """
    Returns up to max_distractors rhyming words (no correct answer),
    with punctuation reattached, excluding:

      - any whose *core* is in forbidden_core_set (existing poem words)
      - uncommon words (by Brown corpus frequency)
      - words with non-letter/quote characters (no 'est.', 'p', 'z.' etc.)
      - words whose coarse POS != target_pos (if target_pos is given)
      - (optionally) words with different syllable count

    Uses phones_for_word + rhyming_part + search() for nicer matches
    tied to a specific pronunciation.
    """
    if not core:
        return []

    if forbidden_core_set is None:
        forbidden_core_set = set()

    # 1) Get pronunciations for the core word
    phones_list = pronouncing.phones_for_word(core.lower())
    if not phones_list:
        return []

    phones = phones_list[0]

    # 2) Get syllable count for the core word
    try:
        core_syllables = pronouncing.syllable_count(phones)
    except Exception:
        core_syllables = None

    # 3) Get rhyme pattern and search for rhyming words
    rhyme_pat = pronouncing.rhyming_part(phones)
    candidates = pronouncing.search(rhyme_pat + "$")

    # Fallback to simple rhymes() if search gives nothing
    if not candidates:
        candidates = pronouncing.rhymes(core.lower())

    cleaned = []
    for w in candidates:
        lw_core = word_core_lower(w)

        # --- basic shape / cleanliness filters ---
        # must be at least 3 letters in core
        if len(lw_core) < 3:
            continue

        # candidate itself must be purely letters / apostrophes
        if not re.fullmatch(r"[A-Za-z']+", w):
            continue

        # skip if already in poem
        if lw_core in forbidden_core_set:
            continue

        # skip uncommon words based on Brown corpus
        if not is_common_word(w, min_frequency=MIN_FREQ):
            continue

        # POS filter: must match coarse POS of target word (if provided)
        if target_pos is not None:
            cand_pos = get_coarse_pos(w)
            if cand_pos is None or cand_pos != target_pos:
                continue

        # optional: require same syllable count
        if core_syllables is not None:
            w_phones = pronouncing.phones_for_word(w.lower())
            if not w_phones:
                continue
            if pronouncing.syllable_count(w_phones[0]) != core_syllables:
                continue

        cleaned.append(w)

    # match capitalization pattern of original core
    if core.istitle():
        cleaned = [w.title() for w in cleaned]

    # reattach punctuation from the original last word
    cleaned = [w + punct for w in cleaned]

    random.shuffle(cleaned)
    return cleaned[:max_distractors]


# --------- main pipeline ---------
def build_line_level_game_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Input df must have columns (your current setup):
        - name       (author name)
        - title      (poem title)
        - poem_link
        - first_line (using this as poem text for now; can switch to poem_text later)

    Output: line-level dataframe with columns:
        poem_slug, author_name, poem_title, poem_link,
        line_index, line_text,
        correct_last_word, choice_1..choice_6, no_choices
    """
    game_rows = []

    for _, row in df.iterrows():
        poem_text = row.get("first_line")
        if not isinstance(poem_text, str) or not poem_text.strip():
            continue  # skip missing/empty

        author_name = row.get("name", "")
        poem_title = row["title"]
        poem_link = row["poem_link"]

        poem_slug = slugify(f"{author_name}-{poem_title}")

        # split into non-empty lines
        lines = [ln for ln in poem_text.split("\n") if ln.strip()]

        # ---- all word cores in this poem (forbidden for distractors) ----
        forbidden_cores = set()
        for ln in lines:
            for tok in ln.split():
                forbidden_cores.add(word_core_lower(tok))

        for line_index, line in enumerate(lines):
            tokens = line.split()
            if not tokens:
                continue

            last_word_raw = tokens[-1]
            core, punct = split_core_and_punct(last_word_raw)

            # POS of the correct word (by core)
            target_pos = get_coarse_pos(core)

            # get up to 5 rhyming distractors with same POS
            distractors = rhyme_distractors(
                core,
                punct,
                forbidden_core_set=forbidden_cores,
                max_distractors=5,
                target_pos=target_pos,
            )

            correct_last_word = last_word_raw  # keep original incl. punctuation
            correct_core_l = word_core_lower(correct_last_word)

            # ---------- HARD NO-DUPLICATE GUARD (core-based) ----------
            unique_distractors = []
            seen_cores = {correct_core_l}  # start with correct word's core

            for d in distractors:
                d_core_l = word_core_lower(d)
                if d_core_l in seen_cores:
                    continue
                seen_cores.add(d_core_l)
                unique_distractors.append(d)

            distractors = unique_distractors
            # ----------------------------------------------------------

            # If there are no distractors at all, keep the line but mark no_choices
            if len(distractors) == 0:
                options = ["", "", "", "", "", ""]  # all blanks for frontend
                no_choices = True
            else:
                # normal case: correct + distractors
                options = [correct_last_word] + distractors
                random.shuffle(options)
                # pad to exactly 6 options
                while len(options) < 6:
                    options.append("")
                no_choices = False

            game_rows.append({
                "poem_slug": poem_slug,
                "author_name": author_name,
                "poem_title": poem_title,
                "poem_link": poem_link,
                "line_index": line_index,
                "line_text": line,
                "correct_last_word": correct_last_word,
                "choice_1": options[0],
                "choice_2": options[1],
                "choice_3": options[2],
                "choice_4": options[3],
                "choice_5": options[4],
                "choice_6": options[5],
                "no_choices": no_choices,
            })

    return pd.DataFrame(game_rows)


# --------- example usage ---------
# import nltk
# nltk.download("brown")
# nltk.download("averaged_perceptron_tagger")  # or 'averaged_perceptron_tagger_eng'
# df = pd.read_csv("poems_with_text.csv")
game_df = build_line_level_game_df(df)
game_df.to_csv("poem_rhyme_game.csv", index=False)


In [83]:
nltk.download("wordnet")

# depending on your NLTK version, one of these will work:
nltk.download("averaged_perceptron_tagger")
nltk.download("averaged_perceptron_tagger_eng")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/melwalsh/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/melwalsh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/melwalsh/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [91]:
import pandas as pd
import random
import re
import pronouncing

from nltk.corpus import brown, wordnet as wn
from nltk import pos_tag, word_tokenize
from collections import Counter

random.seed(42)


# ---------------- CONFIG ----------------
MIN_FREQ = 5                 # common word threshold (Brown)
ENFORCE_POS = True            # turn off if too strict
ENFORCE_WORDNET_VERB = False   # extra guardrail for verbs only
MAX_DISTRACTORS = 5           # +1 correct = 6 total choices
# ----------------------------------------


# --------- frequency helpers ---------
def word_core_lower(word: str) -> str:
    m = re.search(r"[A-Za-z']+", word)
    if m:
        return m.group(0).lower()
    return word.lower()


word_freq = Counter(w.lower() for w in brown.words())

def is_common_word(word: str, min_frequency: int = MIN_FREQ) -> bool:
    core = word_core_lower(word)
    if len(core) < 3:
        return False
    return word_freq.get(core, 0) >= min_frequency


# --------- POS helpers ---------
def coarse_pos(tag: str):
    if tag.startswith("N"):
        return "NOUN"
    if tag.startswith("V"):
        return "VERB"
    if tag.startswith("J"):
        return "ADJ"
    if tag.startswith("R"):
        return "ADV"
    return "OTHER"


def get_last_word_pos_from_line(line: str):
    """
    Tag the last alphabetic word in the line using a simple regex tokenizer.
    Avoids tagging punctuation like ':' as the last 'word'.
    """
    # grab only "word-like" tokens (letters/apostrophes)
    tokens = re.findall(r"[A-Za-z']+", line)
    if not tokens:
        return None

    # tag in context: pos_tag uses the whole sequence
    tagged = pos_tag(tokens)
    last_tag = tagged[-1][1]
    return coarse_pos(last_tag)



def has_wordnet_verb_sense(word: str) -> bool:
    return any(s.pos() == 'v' for s in wn.synsets(word_core_lower(word)))


# --------- misc helpers ---------
def slugify(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^\w\s-]", "", s)
    s = re.sub(r"\s+", "-", s).strip("-")
    return s


def split_core_and_punct(word: str):
    word = word.strip()
    m = re.match(r"^([A-Za-z']+)([^A-Za-z']*)$", word)
    if m:
        return m.group(1), m.group(2)
    return word, ""


# --------- rhyme distractors ----------
def rhyme_distractors(core: str,
                      punct: str,
                      target_pos: str,
                      forbidden_core_set=None,
                      max_distractors: int = MAX_DISTRACTORS):

    if not core:
        return []

    if forbidden_core_set is None:
        forbidden_core_set = set()

    phones_list = pronouncing.phones_for_word(core.lower())
    if not phones_list:
        return []

    phones = phones_list[0]

    # syllables of the correct answer
    try:
        core_syllables = pronouncing.syllable_count(phones)
    except Exception:
        core_syllables = None

    rhyme_pat = pronouncing.rhyming_part(phones)
    candidates = pronouncing.search(rhyme_pat + "$")

    if not candidates:
        candidates = pronouncing.rhymes(core.lower())

    filtered = []

    for w in candidates:
        lw_core = word_core_lower(w)

        # letters-only
        if not re.fullmatch(r"[A-Za-z']+", w):
            continue

        # length reasonable
        if len(lw_core) < 3:
            continue

        # not already in poem
        if lw_core in forbidden_core_set:
            continue

        # common word
        if not is_common_word(w):
            continue

        # match syllable count
        if core_syllables is not None:
            wp = pronouncing.phones_for_word(w.lower())
            if not wp:
                continue
            if pronouncing.syllable_count(wp[0]) != core_syllables:
                continue

        # ---------- POS ENFORCEMENT ----------
        if ENFORCE_POS and target_pos is not None:
            tagged = pos_tag([w])
            cand_pos = coarse_pos(tagged[0][1])

            # keep only same coarse POS
            if cand_pos != target_pos:
                continue

        # extra verb sanity check
        if ENFORCE_WORDNET_VERB and target_pos == "VERB":
            if not has_wordnet_verb_sense(w):
                continue

        filtered.append(w)

    # match capitalization
    if core.istitle():
        filtered = [w.title() for w in filtered]

    filtered = [w + punct for w in filtered]

    random.shuffle(filtered)
    return filtered[:max_distractors]


# -------- MAIN PIPELINE ---------
def build_line_level_game_df(df: pd.DataFrame) -> pd.DataFrame:

    game_rows = []

    for _, row in df.iterrows():

        poem_text = row.get("first_line")
        if not isinstance(poem_text, str) or not poem_text.strip():
            continue

        author_name = row.get("name", "")
        poem_title = row["title"]
        poem_link = row["poem_link"]

        poem_slug = slugify(f"{author_name}-{poem_title}")

        lines = [ln for ln in poem_text.split("\n") if ln.strip()]

        # words already in poem
        forbidden_cores = set()
        for ln in lines:
            for tok in ln.split():
                forbidden_cores.add(word_core_lower(tok))

        for line_index, line in enumerate(lines):

            tokens = line.split()
            if not tokens:
                continue

            last_word_raw = tokens[-1]
            core, punct = split_core_and_punct(last_word_raw)

            # POS in context (important)
            target_pos = get_last_word_pos_from_line(line)

            distractors = rhyme_distractors(
                core=core,
                punct=punct,
                target_pos=target_pos,
                forbidden_core_set=forbidden_cores,
                max_distractors=MAX_DISTRACTORS,
            )

            correct_last_word = last_word_raw

            # remove accidental duplicates
            seen = {word_core_lower(correct_last_word)}
            unique = []
            for d in distractors:
                dc = word_core_lower(d)
                if dc in seen:
                    continue
                seen.add(dc)
                unique.append(d)

            distractors = unique

            if len(distractors) == 0:
                options = [""] * 6
                no_choices = True
            else:
                options = [correct_last_word] + distractors
                random.shuffle(options)
                while len(options) < 6:
                    options.append("")
                no_choices = False

            game_rows.append({
                "poem_slug": poem_slug,
                "author_name": author_name,
                "poem_title": poem_title,
                "poem_link": poem_link,
                "line_index": line_index,
                "line_text": line,
                "correct_last_word": correct_last_word,
                "choice_1": options[0],
                "choice_2": options[1],
                "choice_3": options[2],
                "choice_4": options[3],
                "choice_5": options[4],
                "choice_6": options[5],
                "no_choices": no_choices,
                "target_pos": target_pos,
            })

    return pd.DataFrame(game_rows)


# -------- run it --------
# import nltk
# nltk.download("brown")
# nltk.download("punkt")
# nltk.download("averaged_perceptron_tagger")
# nltk.download("wordnet")

# df = pd.read_csv("poems_with_text.csv")
game_df = build_line_level_game_df(df)
game_df.to_csv("poem_rhyme_game.csv", index=False)


In [86]:
nltk.download("punkt")
nltk.download("punkt_tab")  # only if error mentions punkt_tab


[nltk_data] Downloading package punkt to /Users/melwalsh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/melwalsh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [74]:
import pandas as pd
import random
import re
import pronouncing

random.seed(42)  # reproducible builds


# --------- helpers ---------
def slugify(s: str) -> str:
    """Make a URL / id-friendly slug from a string."""
    s = s.lower()
    s = re.sub(r"[^\w\s-]", "", s)  # remove punctuation
    s = re.sub(r"\s+", "-", s).strip("-")  # spaces -> hyphens
    return s


def split_core_and_punct(word: str):
    """
    Split token into alphabetic core + trailing punctuation.
    Examples:
        'night,'    -> ('night', ',')
        'Sea!'      -> ('Sea', '!')
        'breast:'   -> ('breast', ':')
        '"time."'   -> ('time', '."')
    """
    word = word.strip()
    m = re.match(r"^([A-Za-z']+)([^A-Za-z']*)$", word)
    if m:
        return m.group(1), m.group(2)
    else:
        return word, ""


def word_core_lower(word: str) -> str:
    """
    Very forgiving core extractor for duplicate checks:
    - takes the first run of letters/apostrophes
    - lowercases it
    So 'breast:', 'Breast', '"breast:"' -> 'breast'
    """
    m = re.search(r"[A-Za-z']+", word)
    if m:
        return m.group(0).lower()
    return word.lower()


def rhyme_distractors(core: str,
                      punct: str,
                      forbidden_core_set=None,
                      max_distractors: int = 5):
    """
    Returns up to max_distractors rhyming words (no correct answer),
    with punctuation reattached, excluding:

      - any whose *core* is in forbidden_core_set (e.g. existing words in poem)

    Uses phones_for_word + rhyming_part + search() for nicer matches
    tied to a specific pronunciation.
    """
    if not core:
        return []

    if forbidden_core_set is None:
        forbidden_core_set = set()

    # 1) Get pronunciations for the core word
    phones_list = pronouncing.phones_for_word(core.lower())
    if not phones_list:
        return []

    phones = phones_list[0]

    # 2) Get syllable count for the core word
    try:
        core_syllables = pronouncing.syllable_count(phones)
    except Exception:
        core_syllables = None

    # 3) Get rhyme pattern and search for rhyming words
    rhyme_pat = pronouncing.rhyming_part(phones)
    candidates = pronouncing.search(rhyme_pat + "$")

    # Fallback to simple rhymes() if search gives nothing
    if not candidates:
        candidates = pronouncing.rhymes(core.lower())

    cleaned = []
    for w in candidates:
        lw_core = word_core_lower(w)

        # skip if already in poem
        if lw_core in forbidden_core_set:
            continue

        # optional: require same syllable count
        if core_syllables is not None:
            w_phones = pronouncing.phones_for_word(w.lower())
            if not w_phones:
                continue
            if pronouncing.syllable_count(w_phones[0]) != core_syllables:
                continue

        cleaned.append(w)

    # match capitalization pattern of original core
    if core.istitle():
        cleaned = [w.title() for w in cleaned]

    # reattach punctuation
    cleaned = [w + punct for w in cleaned]

    random.shuffle(cleaned)
    return cleaned[:max_distractors]


# --------- main pipeline ---------
def build_line_level_game_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Input df must have columns (your current setup):
        - name       (author name)
        - title      (poem title)
        - poem_link
        - first_line (using this as poem text for now; can switch to poem_text later)

    Output: line-level dataframe with columns:
        poem_slug, author_name, poem_title, poem_link,
        line_index, line_text,
        correct_last_word, choice_1..choice_6
    """
    game_rows = []

    for _, row in df.iterrows():
        poem_text = row.get("first_line")
        if not isinstance(poem_text, str) or not poem_text.strip():
            continue  # skip missing/empty

        author_name = row.get("name", "")
        poem_title = row["title"]
        poem_link = row["poem_link"]

        poem_slug = slugify(f"{author_name}-{poem_title}")

        # split into non-empty lines
        lines = [ln for ln in poem_text.split("\n") if ln.strip()]

        # ---- all word cores in this poem (forbidden for distractors) ----
        forbidden_cores = set()
        for ln in lines:
            for tok in ln.split():
                forbidden_cores.add(word_core_lower(tok))

        for line_index, line in enumerate(lines):
            tokens = line.split()
            if not tokens:
                continue

            last_word_raw = tokens[-1]
            core, punct = split_core_and_punct(last_word_raw)

            # get up to 5 rhyming distractors
            distractors = rhyme_distractors(
                core,
                punct,
                forbidden_core_set=forbidden_cores,
                max_distractors=5,
            )

            correct_last_word = last_word_raw  # keep original incl. punctuation
            correct_core_l = word_core_lower(correct_last_word)

            # ---------- HARD NO-DUPLICATE GUARD (core-based) ----------
            unique_distractors = []
            seen_cores = {correct_core_l}  # start with correct word's core

            for d in distractors:
                d_core_l = word_core_lower(d)
                if d_core_l in seen_cores:
                    continue
                seen_cores.add(d_core_l)
                unique_distractors.append(d)

            distractors = unique_distractors
            # ----------------------------------------------------------

            options = [correct_last_word] + distractors

            # if we don't get at least 2 options, skip this line
            if len(options) < 2:
                continue

            random.shuffle(options)

            # pad to exactly 6 options (1 correct + up to 5 distractors)
            while len(options) < 6:
                options.append("")

            game_rows.append({
                "poem_slug": poem_slug,
                "author_name": author_name,
                "poem_title": poem_title,
                "poem_link": poem_link,
                "line_index": line_index,
                "line_text": line,
                "correct_last_word": correct_last_word,
                "choice_1": options[0],
                "choice_2": options[1],
                "choice_3": options[2],
                "choice_4": options[3],
                "choice_5": options[4],
                "choice_6": options[5],
            })

    return pd.DataFrame(game_rows)


# --------- example usage ---------
# df = pd.read_csv("poems_with_text.csv")
game_df = build_line_level_game_df(df)
game_df.to_csv("poem_rhyme_game.csv", index=False)


In [65]:
build_line_level_game_df(df)

""


In [44]:
df[['true_words', 'rhyming_words']] = df['first_line'].apply(get_rhymes)

ValueError: Sample larger than population or is negative

In [49]:
sample = ["tree"]

In [50]:
sample[-1]

'tree'

In [62]:
import re
import random
import pronouncing
import string

def get_rhymes(poem):
    lines = poem.split("\n")
    words = [line.split() for line in lines]

    last_words = [line[-1] for line in words if len(line) >= 1]

    rhyming_words = []

    for word in last_words:
        word = word.strip()

        # ---- 1) capture core word + trailing punctuation ----
        # matches:   core="night"   punct=","  in "night,"
        m = re.match(r"^([A-Za-z']+)([^A-Za-z']*)$", word)
        if m:
            core = m.group(1)
            punct = m.group(2)
        else:
            core = word
            punct = ""

        # ---- 2) get rhymes for core word ----
        if core.istitle():
            rhymes = pronouncing.rhymes(core.lower())
            rhymes = [rhyme.title() for rhyme in rhymes if len(rhyme) > 2]
        else:
            rhymes = pronouncing.rhymes(core.lower())
            rhymes = [rhyme for rhyme in rhymes if len(rhyme) > 2]

        if len(rhymes) >= 5:
            rhymes = random.sample(rhymes, 5)

        # ---- 3) reattach punctuation to each rhyme ----
        rhymes_with_punct = [r + punct for r in rhymes]

        rhyming_words.append(rhymes_with_punct)

    return last_words, rhyming_words


In [60]:
def get_rhymes(poem):
    lines = poem.split("\n")
    words = [line.split() for line in lines]
    #for line in words:
     #   print(line[-1])

    
    last_words = [line[-1] for line in words if len(line) >= 1]
    print(last_words)

    rhyming_words = []
    for word in last_words:
        word = word.strip()
        if word.istitle():
            rhymes = pronouncing.rhymes(word) 
            rhymes = [rhyme.title() for rhyme in rhymes if len(rhyme) > 2]
            if len(rhymes) >= 5:
                rhymes = random.sample(rhymes, 5)
        else:
            rhymes = pronouncing.rhymes(word)
            rhymes = [rhyme for rhyme in rhymes if len(rhyme) > 2]
            if len(rhymes) >= 5:
                rhymes = random.sample(rhymes, 5)

        rhyming_words.append(rhymes) 


   # print(words)
    #print(last_words)
    #print(rhyming_words)
    return last_words, rhyming_words

    

    #return last_words, 


In [43]:
get_rhymes("In sixth grade Mrs. Walker\nslapped the back")

[['In', 'sixth', 'grade', 'Mrs.', 'Walker'], ['slapped', 'the', 'back']]
['Walker', 'back']
[['Walker', 'Talker', 'Stalker', 'Hawker', 'Chalker'], ['vanvlack', 'crack', 'brack', 'haque', 'hacke']]


In [6]:
pronouncing.rhymes("climbing")

['diming', 'liming', 'priming', 'rhyming', 'timing']